<a href="https://colab.research.google.com/github/kgpark88/visionai/blob/main/pytorch_iris_classifIer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 붓꽃(Iris) 품종 분류 모델

## 데이터 로드

In [1]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

In [2]:
df = pd.DataFrame(iris.data)
df.columns = iris.feature_names
df['label'] = iris.target

In [3]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


## 입력데이터(input data)와 레이블(label) 분리

In [4]:
X = df.drop('label', axis=1).to_numpy()
Y = df['label'].to_numpy().reshape((-1,1))

In [5]:
X.shape

(150, 4)

In [6]:
Y.shape

(150, 1)

## 입력 데이터 정규화

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## 훈련데이터셋과 테스트데이터셋으로 분리

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

## 데이터셋 클래스

In [9]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

class TensorData(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.LongTensor(y_data)
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

## 데이터 로더

In [10]:
train_ds = TensorData(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)

test_ds = TensorData(X_test, Y_test)
testloader = torch.utils.data.DataLoader(test_ds, batch_size=16, shuffle=False)

In [11]:
test_ds[0]

(tensor([ 0.3110, -0.5924,  0.5354,  0.0009]), tensor(1))

## 신경망 모델 아키텍처

In [12]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(4, 200)
        self.fc2 = nn.Linear(200, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)
        return X

## 신경망 모델 객체 생성

In [13]:
model = Classifier()

## 손실 함수

In [14]:
criterion = nn.CrossEntropyLoss()

## 옵티마이저를 생성하고, 생성한 모델의 파라미터를  최적화 대상으로 등록


In [15]:
 optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-7)

## 신경망 모델 훈련(학습)

In [16]:
epochs = 10

for epoch in range(epochs):
    print(f'### epoch {epoch+1} ###############')
    for i, data in enumerate(trainloader, 0):
        input, target = data
        optimizer.zero_grad()
        pred = model(input)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        print(f'{i+1} : Loss {loss}')

### epoch 1 ###############
1 : Loss 1.115409016609192
2 : Loss 1.0722088813781738
3 : Loss 1.061854600906372
4 : Loss 1.0023988485336304
5 : Loss 0.9306171536445618
6 : Loss 0.9811887145042419
7 : Loss 0.9298067092895508
### epoch 2 ###############
1 : Loss 0.9043833613395691
2 : Loss 0.85558021068573
3 : Loss 0.867655873298645
4 : Loss 0.8547680377960205
5 : Loss 0.832449734210968
6 : Loss 0.7473430633544922
7 : Loss 0.831342339515686
### epoch 3 ###############
1 : Loss 0.6943833231925964
2 : Loss 0.7519823908805847
3 : Loss 0.7216482758522034
4 : Loss 0.8271784782409668
5 : Loss 0.7789612412452698
6 : Loss 0.834022581577301
7 : Loss 0.7820742726325989
### epoch 4 ###############
1 : Loss 0.6742343902587891
2 : Loss 0.7957671284675598
3 : Loss 0.6850881576538086
4 : Loss 0.6831865310668945
5 : Loss 0.6893842220306396
6 : Loss 0.7587785720825195
7 : Loss 0.8075346946716309
### epoch 5 ###############
1 : Loss 0.770237386226654
2 : Loss 0.6666763424873352
3 : Loss 0.7329469323158264
4

## 신경망 모델 평가

In [17]:
correct = 0
with torch.no_grad():
  for i, data in enumerate(test_ds):
    label = data[1].numpy()
    output = model.forward(data[0].reshape(1,-1))
    pred = output.argmax().item()

    if label == pred:
        correct += 1

print(f'정확도 : {correct/len(test_ds)*100:.2f}%')

정확도 : 95.56%
